In [ ]:
!pip install keras_tuner -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from keras.metrics import categorical_accuracy
import keras_tuner as kt
import tensorflow as tf
import glob
import os


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# merge the files
disease_path = "/content/drive/My Drive/Colab Notebooks/Disease_symptom_and_patient_profile_dataset.csv"

# Concatinate the files
disease_df = pd.read_csv(disease_path,encoding="utf-8")

# Review the DataFrame
disease_df

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
...,...,...,...,...,...,...,...,...,...,...
344,Stroke,Yes,No,Yes,No,80,Female,High,High,Positive
345,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive
346,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive
347,Stroke,Yes,No,Yes,No,90,Female,High,High,Positive


In [ ]:
#identify the data types in the file before transforming
disease_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Disease               349 non-null    object
 1   Fever                 349 non-null    object
 2   Cough                 349 non-null    object
 3   Fatigue               349 non-null    object
 4   Difficulty Breathing  349 non-null    object
 5   Age                   349 non-null    int64 
 6   Gender                349 non-null    object
 7   Blood Pressure        349 non-null    object
 8   Cholesterol Level     349 non-null    object
 9   Outcome Variable      349 non-null    object
dtypes: int64(1), object(9)
memory usage: 27.4+ KB


In [ ]:
disease_df.nunique()

Disease                 116
Fever                     2
Cough                     2
Fatigue                   2
Difficulty Breathing      2
Age                      26
Gender                    2
Blood Pressure            3
Cholesterol Level         3
Outcome Variable          2
dtype: int64

In [ ]:
#initiate scaling
disease_data_scaled = StandardScaler().fit_transform(disease_df[["Age"]])
#create df with scaled data
disease_scaled_df = pd.DataFrame(disease_data_scaled,columns=["Age"])
#show the newly scaled df
disease_scaled_df.head()

,Age
0,-2.091160
1,-1.631964
2,-1.631964
3,-1.631964
4,-1.631964


In [ ]:
hot_df = pd.get_dummies(disease_df[['Gender','Difficulty Breathing','Fever','Cough','Fatigue','Blood Pressure','Cholesterol Level']])
hot_df

,Gender_Female,Gender_Male,Difficulty Breathing_No,Difficulty Breathing_Yes,Fever_No,Fever_Yes,Cough_No,Cough_Yes,Fatigue_No,Fatigue_Yes,Blood Pressure_High,Blood Pressure_Low,Blood Pressure_Normal,Cholesterol Level_High,Cholesterol Level_Low,Cholesterol Level_Normal
0,1,0,0,1,0,1,1,0,0,1,0,1,0,0,0,1
1,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1
2,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1
3,0,1,0,1,0,1,0,1,1,0,0,0,1,0,0,1
4,0,1,0,1,0,1,0,1,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,1,0,1,0,0,1,1,0,0,1,1,0,0,1,0,0
345,0,1,1,0,0,1,1,0,0,1,1,0,0,1,0,0
346,0,1,1,0,0,1,1,0,0,1,1,0,0,1,0,0
347,1,0,1,0,0,1,1,0,0,1,1,0,0,1,0,0


In [ ]:
label_encoder = preprocessing.LabelEncoder()
disease_df["Disease"]=label_encoder.fit_transform(disease_df["Disease"])
disease_df["Disease"].unique()

array([ 56,  24,  37,   6,  53,   1,   4,  32,  42,  81,  92,  31,  60,
       101, 112,  30,  48,  57,  69,  72,  98, 111,   9,  13,  15,  23,
        52,  70,  73, 113,  77,  84,   7,  21,  39,  51,  74,  90,  93,
        22,  25,  61,  65, 100,  58,  75,  59,   0,  11,  28,  43,  91,
        16,  26,  35,  41,  46,  54,  64, 108,  62,  55,   8,  27,  50,
        68,  79, 109, 115,  14,  83,  95,  97, 104,   3,  18,  38,  96,
        99,  34,  36,  63,  80,  85,   5,  40,  45,  66,  82,  47,  87,
       102, 110,  12,  67,  76,  86,  19,  49,  88, 105,  10,  78, 107,
         2,  20,  29,  33,  71,  17,  89,  94,  44, 103, 106, 114])

In [ ]:
disease_concat_df = pd.concat([disease_df["Disease"],disease_scaled_df,hot_df],axis=1)
disease_concat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Disease                   349 non-null    int64  
 1   Age                       349 non-null    float64
 2   Gender_Female             349 non-null    uint8  
 3   Gender_Male               349 non-null    uint8  
 4   Difficulty Breathing_No   349 non-null    uint8  
 5   Difficulty Breathing_Yes  349 non-null    uint8  
 6   Fever_No                  349 non-null    uint8  
 7   Fever_Yes                 349 non-null    uint8  
 8   Cough_No                  349 non-null    uint8  
 9   Cough_Yes                 349 non-null    uint8  
 10  Fatigue_No                349 non-null    uint8  
 11  Fatigue_Yes               349 non-null    uint8  
 12  Blood Pressure_High       349 non-null    uint8  
 13  Blood Pressure_Low        349 non-null    uint8  
 14  Blood Pres

In [ ]:
y = disease_concat_df["Disease"].values
X = disease_concat_df.drop(["Disease"],1).values
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

<ipython-input-11-0f501a1b5afd>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = disease_concat_df.drop(["Disease"],1).values


In [ ]:
disease_scaler = StandardScaler()
X_scaler = disease_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
len(X_train)
# X_train_shaped = X_train_scaled.reshape()

261

In [ ]:
disease_input_features = len(X_train[0])
hidden_nodes_layer1 =  50
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 4
hidden_nodes_layer4 = 2

disease_nn = tf.keras.models.Sequential()

# disease_nn.add(
#     tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=disease_input_features, activation="ReLU")
# )

disease_nn.add(
    tf.keras.layers.InputLayer(input_shape=(17))
)

disease_nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="ReLU"))

# Second hidden layer
# disease_nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="ReLU"))

# # Third layer
# disease_nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="ReLU"))

# # Fourth layer
# disease_nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="ReLU"))

#Output layer
disease_nn.add(tf.keras.layers.Dense(units=1, activation="softmax"))


# Check the structure of the model
disease_nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                900       
                                                                 
 dense_1 (Dense)             (None, 30)                1530      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 2,461
Trainable params: 2,461
Non-trainable params: 0
_________________________________________________________________


In [ ]:
disease_nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["categorical_accuracy","accuracy"])

In [ ]:
print(y_train)

[ 51  27  13  92  90  37 101  20  99  51 101  84  42  26   2  89  60  90
  31  32  31 112  77 110  98  77  60   4  77  32  77  48   6  57  51  59
  31  42  92  56  69  59   6  58  53  53  81  83  92 109  60  17  92  84
  92 112   4  77   1 101  84  30   9  24   6  74  67  49   6  56  70  65
  58  18  77  50  77  58  24  13   5  26  13   1  13  14   0   6  82  43
  62  44  77  20  48  76  71 112 101  37  28  40  75  75  91   6  95 105
  16 113  77  69  57  49  32  37  27  46  27  82  52  62  51  51  61  31
  90  84   4   6  69  79 108  62  96  41  32  77 111  87 115  55   6  15
  69  82   3  64  48 114 108  51   6  56   2  82  29  56  34  70  88  58
  32  61  81  31   6  51  21  70 111 104   2  57  26  42  47  27  82  90
  24  54  16  53  39  32  55  13  60  77  55 108  85   4  31  32  61 103
  33 102 104   8  37  57  55  26 101  69  24 113  42   1   6  75  92  24
  77  70  24  69  84  57 101  53  78  94  13  84   6 109  51 101  32  69
   6  73  80  66 112   2   1  26  61  51   4 101  6

In [ ]:
disease_fit_model = disease_nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


9/9 [==============================] - 4s 11ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.0153
Epoch 2/25
9/9 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.0153
Epoch 3/25
9/9 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.0153
Epoch 4/25
9/9 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.0153
Epoch 5/25
9/9 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.0153
Epoch 6/25
9/9 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.0153
Epoch 7/25
9/9 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.0153
Epoch 8/25
9/9 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - categor

In [ ]:
model_loss, model_accuracy = disease_nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3/3 - 0s - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - accuracy: 0.0227 - 28ms/epoch - 9ms/step


ValueError: ignored

In [ ]:
def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.InputLayer(input_shape=(17)))
  hidden_nodes_layer1 =  16
  hidden_nodes_layer2 = 8
  hidden_nodes_layer3 = 4
  hidden_nodes_layer4 = 2
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(tf.keras.layers.Dense(units=hp_units, activation='ReLU'))
  model.add(tf.keras.layers.Dense(hidden_nodes_layer1))
  model.add(tf.keras.layers.Dense(hidden_nodes_layer2))
  model.add(tf.keras.layers.Dense(hidden_nodes_layer3))
  model.add(tf.keras.layers.Dense(hidden_nodes_layer4))
  model.add(tf.keras.layers.Dense(1,activation="softmax"))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
project_tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=25,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
project_tuner.search(X_train_scaled, y_train, epochs=25, validation_split=0.2)

Trial 2 Complete [00h 00m 02s]

Best val_accuracy So Far: None
Total elapsed time: 00h 00m 06s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
192               |160               |units
0.01              |0.001             |learning_rate
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/3


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/tuners/hyperband.py", line 425, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/hypermodel.py", line

RuntimeError: ignored